In [51]:
import json
import pandas as pd
from pathlib import Path
from api_calls import get_multiple_api_calls_given_sentiment
import regex as re

In [52]:
TRAINING_DIR = Path("./data/training")

In [53]:
with open(TRAINING_DIR/"training_data_prompt_0.jsonl", "r") as file:
    training_data = list(file)

training_data_json = [json.loads(ele) for ele in training_data]

In [54]:
training_data_df = pd.read_csv(TRAINING_DIR/"training_data.csv", usecols=["sentence", "label"])

In [55]:
json_sentences = [ele["messages"][1]["content"] for ele in training_data_json]

In [56]:
sentences_to_add = []
labels = []
for i in range(training_data_df.shape[0]):
    if training_data_df.iloc[i, 0] not in json_sentences:
        sentences_to_add.append(training_data_df.iloc[i,0])
        labels.append(training_data_df.iloc[i,1])

In [7]:
reasons = await get_multiple_api_calls_given_sentiment(labels, sentences_to_add)

2023-10-19 10:41:18 [info     ] 200
2023-10-19 10:41:18 [info     ] completed 1 / 1
2023-10-19 10:41:19 [info     ] 200
2023-10-19 10:41:19 [info     ] completed 1 / 1
2023-10-19 10:41:19 [info     ] 200
2023-10-19 10:41:19 [info     ] completed 1 / 1
2023-10-19 10:41:19 [info     ] 200
2023-10-19 10:41:19 [info     ] completed 1 / 1
2023-10-19 10:41:20 [info     ] 200
2023-10-19 10:41:20 [info     ] completed 1 / 1
2023-10-19 10:41:22 [info     ] 200
2023-10-19 10:41:22 [info     ] completed 1 / 1
2023-10-19 10:41:22 [info     ] 200
2023-10-19 10:41:22 [info     ] completed 1 / 1
2023-10-19 10:41:22 [info     ] 200
2023-10-19 10:41:22 [info     ] completed 1 / 1
2023-10-19 10:41:22 [info     ] 200
2023-10-19 10:41:22 [info     ] completed 1 / 1
2023-10-19 10:41:22 [info     ] 200
2023-10-19 10:41:22 [info     ] completed 1 / 1
2023-10-19 10:41:22 [info     ] 200
2023-10-19 10:41:22 [info     ] completed 1 / 1
2023-10-19 10:41:22 [info     ] 200
2023-10-19 10:41:22 [info     ] complete

In [10]:
for sentence, reason in zip(sentences_to_add, reasons):
    training_data_json.append(
            {
                "messages": 
                [
                    {
                        "role": "system", 
                        "content": "you are an economist. label the following sentence as hawkish, dovish or neutral and provide a reason for your label."
                    },
                    {
                        "role": "user",
                        "content": sentence
                    },
                    {
                        "role": "assistant",
                        "content": reason
                    }
                ]
            }
        )

In [35]:
len(sentences_to_add)

0

In [57]:
training_data_json

[{'messages': [{'role': 'system',
    'content': "You are an expert sentence classifier. Classify the following sentence from FOMC into 'HAWKISH', 'DOVISH', or 'NEUTRAL' class. Label 'HAWKISH' if it corresponds to a tightening of the monetary policy, 'DOVISH' if it corresponds to an easing of the monetary policy, or 'NEUTRAL' if the stance is neutral. Your reply should follow the format 'This sentence is `LABEL` because..."},
   {'role': 'user',
    'content': 'But monetary policy has little ability to affect inflation this year.'},
   {'role': 'assistant',
    'content': "This sentence is neutral because it neither supports nor criticizes US monetary policy. Instead, it simply states a fact about the policy's limited impact on inflation this year."}]},
 {'messages': [{'role': 'system',
    'content': "You are an expert sentence classifier. Classify the following sentence from FOMC into 'HAWKISH', 'DOVISH', or 'NEUTRAL' class. Label 'HAWKISH' if it corresponds to a tightening of the mo

In [61]:
for data in training_data_json:
    data["messages"][0]["content"] = "You are an expert sentence classifier. Classify the following sentence from FOMC into 'HAWKISH', 'DOVISH', or 'NEUTRAL' class. Label 'HAWKISH' if it corresponds to a tightening of the monetary policy, 'DOVISH' if it corresponds to an easing of the monetary policy, or 'NEUTRAL' if the stance is neutral. Your reply should follow the format 'This sentence is `LABEL` because...'"

    # data["messages"][2]["content"] = re.search(r"hawkish|dovish|neutral", data["messages"][2]["content"], re.IGNORECASE).group(0).upper()

In [62]:
training_data_json

[{'messages': [{'role': 'system',
    'content': "You are an expert sentence classifier. Classify the following sentence from FOMC into 'HAWKISH', 'DOVISH', or 'NEUTRAL' class. Label 'HAWKISH' if it corresponds to a tightening of the monetary policy, 'DOVISH' if it corresponds to an easing of the monetary policy, or 'NEUTRAL' if the stance is neutral. Your reply should follow the format 'This sentence is `LABEL` because...'"},
   {'role': 'user',
    'content': 'But monetary policy has little ability to affect inflation this year.'},
   {'role': 'assistant',
    'content': "This sentence is neutral because it neither supports nor criticizes US monetary policy. Instead, it simply states a fact about the policy's limited impact on inflation this year."}]},
 {'messages': [{'role': 'system',
    'content': "You are an expert sentence classifier. Classify the following sentence from FOMC into 'HAWKISH', 'DOVISH', or 'NEUTRAL' class. Label 'HAWKISH' if it corresponds to a tightening of the m

In [63]:
with open(TRAINING_DIR/"training_data_prompt_0.jsonl", 'w') as outfile:
    for entry in training_data_json:
        json.dump(entry, outfile)
        outfile.write('\n')

In [65]:
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

In [66]:
openai.File.create(
  file=open(TRAINING_DIR/"training_data_prompt_0.jsonl", "rb"),
  purpose='fine-tune'
)

<File file id=file-4FrkmT8rPk9ZxLfhvumQtyjj at 0x168316340> JSON: {
  "object": "file",
  "id": "file-4FrkmT8rPk9ZxLfhvumQtyjj",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 2278310,
  "created_at": 1697712193,
  "status": "uploaded",
  "status_details": null
}

In [67]:
openai.FineTuningJob.create(training_file="file-4FrkmT8rPk9ZxLfhvumQtyjj", model="gpt-3.5-turbo")


<FineTuningJob fine_tuning.job id=ftjob-bcx6Jfr5ikn4GvjrahflGjCR at 0x16817db20> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-bcx6Jfr5ikn4GvjrahflGjCR",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1697712230,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-I4j7CvqT1sUoMLq85ywq27zV",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-4FrkmT8rPk9ZxLfhvumQtyjj",
  "hyperparameters": {
    "n_epochs": "auto"
  },
  "trained_tokens": null,
  "error": null
}

In [68]:
openai.File.create(
  file=open(TRAINING_DIR/"training_data_prompt_2.jsonl", "rb"),
  purpose='fine-tune'
)

<File file id=file-jgWWr25JnDOck3fWwurToLAI at 0x13b39f2e0> JSON: {
  "object": "file",
  "id": "file-jgWWr25JnDOck3fWwurToLAI",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 1680880,
  "created_at": 1697712254,
  "status": "uploaded",
  "status_details": null
}

In [69]:
openai.FineTuningJob.create(training_file="file-jgWWr25JnDOck3fWwurToLAI", model="gpt-3.5-turbo")

<FineTuningJob fine_tuning.job id=ftjob-uFFZb7DaryrOi4GydArDPA2H at 0x16830eed0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-uFFZb7DaryrOi4GydArDPA2H",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1697712283,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-I4j7CvqT1sUoMLq85ywq27zV",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-jgWWr25JnDOck3fWwurToLAI",
  "hyperparameters": {
    "n_epochs": "auto"
  },
  "trained_tokens": null,
  "error": null
}

In [77]:
openai.FineTuningJob.list(limit=10)

<OpenAIObject list at 0x168339df0> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job",
      "id": "ftjob-uFFZb7DaryrOi4GydArDPA2H",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1697712283,
      "finished_at": 1697718369,
      "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::8BMOIo6W",
      "organization_id": "org-I4j7CvqT1sUoMLq85ywq27zV",
      "result_files": [
        "file-LAKv2C1jnMzcUMrcn0pcqIF6"
      ],
      "status": "succeeded",
      "validation_file": null,
      "training_file": "file-jgWWr25JnDOck3fWwurToLAI",
      "hyperparameters": {
        "n_epochs": 3
      },
      "trained_tokens": 1080375,
      "error": null
    },
    {
      "object": "fine_tuning.job",
      "id": "ftjob-bcx6Jfr5ikn4GvjrahflGjCR",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1697712230,
      "finished_at": 1697718251,
      "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::8BMMOD86",
      "organization_id": "org-I4j7CvqT1sUoM

In [79]:
from datetime import datetime as dt

In [84]:
dt.fromtimestamp(1697712283)

datetime.datetime(2023, 10, 19, 11, 44, 43)

In [85]:
dt.fromtimestamp(1697712230)


datetime.datetime(2023, 10, 19, 11, 43, 50)

In [86]:
pd.read_parquet("./data/scraping/all_minutes.parquet")

,Federal_Reserve_Mins
1995-02-01,A meeting of the Federal Open Market Committee...
1995-03-28,A meeting of the Federal Open Market Committee...
1995-05-23,A meeting of the Federal Open Market Committee...
1995-07-06,A meeting of the Federal Open Market Committee...
1995-08-22,A meeting of the Federal Open Market Committee...
...,...
2023-03-22,"The Federal Reserve, the central bank of the U..."
2023-05-03,"The Federal Reserve, the central bank of the U..."
2023-06-14,"The Federal Reserve, the central bank of the U..."
2023-07-26,"The Federal Reserve, the central bank of the U..."
